<img src="Imgs/QS.png">

## Overview 
Alice wants to send quantum information to Bob. Specifically, suppose she wants to send the qubit state |$\varphi$> = $\alpha$|0> + $\beta$|1> .This entails passing on information about $\alpha$ and $\beta$ to Bob.

---

### Revision of Quantum communication rules


1- Qubits can be entagelment regardless of physical distance between them.  
2- The entagelment breaks if one of the qubits is measured.  
3- Qubits in arbitrary state can not be cloned.  
4- A qubit with a known state can be copied. 

In [1]:
#Codes in this notebook are inspired by IBM's Qiskit tutorials and the arXiv:1903.04359v1 paper

#Libraries needed to implement and simulate quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute 
import math as m
#Custem functions to simplify answers
import Our_Qiskit_Functions as oq #a part of the libabry presented in arXiv:1903.04359v1.


#Initialize backends simulators to visualize circuits
S_simulator = Aer.backends(name='statevector_simulator')[0]
Q_simulator = Aer.backends(name='qasm_simulator')[0]
sim = Aer.get_backend('aer_simulator')

#Create quantum registers (to hold qubits)
q = QuantumRegister(2)
#Classical register to hold classical bits (Used to measure results)
c = ClassicalRegister(2)
#Create a quantum circuit using the above registers
qc = QuantumCircuit(q,c)

Alice would like to send Bob a qubit that is in some unknown state *but* Alice cannot directly clone the qubit, because of the “No Cloning Theorem”!!  

So, how can she send it??


Alice prepares a pair of entangled qubits and sends one to Bob.

$$
|\psi\rangle=(\alpha|0\rangle+\beta|1\rangle)
$$

$$
\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)
$$

In [9]:
#Code of Equation (3) and (4)
# Create a Quantum Register and classical registers with 3 qubits and 3 classical bits.
q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)

# Prepare an initial state using a single unitary
qc.u1(0.5, q[0])

 # Prepare an entangled pair using qubit ② and qubit ③
qc.h(q[1])
qc.cx(q[1], q[2])
oq.Wavefunction(qc)
# Barrier to prevent gate reordering for optimization
qc.barrier(q)
qc.draw()

┌─────────┐      ░ 
q53_0: ┤ U1(0.5) ├──────░─
       └──┬───┬──┘      ░ 
q53_1: ───┤ H ├─────■───░─
          └───┘   ┌─┴─┐ ░ 
q53_2: ───────────┤ X ├─░─
                  └───┘ ░ 
 c6: 3/═══════════════════

---
The state of the three qubits now is:

$$
\frac{1}{\sqrt{2}}\left[\begin{array}{c}{\alpha|0\rangle(|00\rangle+|11\rangle)+} \\ {\beta|1\rangle(|00\rangle+|11\rangle)}\end{array}\right]
$$

Just like we mentioned in quantum communication, we now apply a *CNOT* gate to the system, between the qubit Alice wants to teleport and the one entageled with Bob's.

$$
\frac{1}{\sqrt{2}}\left[\begin{array}{c}{\alpha|0\rangle(|00\rangle+|11\rangle)+} \\ {\beta|1\rangle(|10\rangle+|01\rangle)}\end{array}\right]
$$    


Alice go ahead and measures the result, giving her 1 or 0.

**How is this information helpful?**   

The result means that the remaining states are:

$$
0 {\longrightarrow} \alpha|00\rangle+\beta|11\rangle
$$

*OR*

$$
1 {\longrightarrow} \alpha|01\rangle+\beta|10\rangle
$$

At this point Alice writes this measurement down and tells Bob what she measured. Both Alice and Bob know that they can flip the state using the X gate.


In [3]:
#Code for the prvious cell
# Perform a CNOT between qubit (1) and qubit (2)
qc.cx(q[0], q[1])

# Measure qubit (2) in the computational basis
qc.measure(q[1], c[1])

# Measure qubit (1) in the |+> and |-> basis
qc.h(q[0])
qc.measure(q[0], c[0])

# If needed Perform a bit flip correction to qubit (3)
if c[1] == 1:
    qc.x(q[2])

---
The next thing that Alice does is to perform a *Hadamard* transformation on her original qubit so that she can tell Bob if he needs to perform a phase flip on his qubit or not. 

Alice now performs a _*measurement*_ after applying the Hadamard gate which enables her to measure a 0 when the qubit is $|+\rangle$ and 1 when the qubit is $|−\rangle$, she then send the results of this measurement to Bob. 


In [4]:
# Measure qubit (1) in the |+> and |-> basis
qc.h(q[0])
qc.measure(q[0], c[0])

# If needed Perform a bit flip correction to qubit (3)
if c[1] == 1:
    qc.x(q[2])

---
*Although* we _DID_ telport a qubit from Alice to Bob, Alice and Bob did not communicate *faster* than the speed of light!!

Alice needed to share her measurement with Bob through some classical channel which is _not_ faster than light!

Alice ended up destroying her qubit in the process of sending it to Bob.

In [5]:
#The complete code for teleportation
#Libraries needed to implement and simulate quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute 
# Create a Quantum Register and classical registers with 3 qubits and 3 classical bits.
q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)
 # Prepare an entangled pair using qubit (2) and qubit (3)
qc.h(q[1])
qc.cx(q[1], q[2])
# Barrier to prevent gate reordering for optimization
qc.barrier(q)
# Perform a CNOT between qubit (1) and qubit (2)
qc.cx(q[0], q[1])
# Measure qubit (2) in the computational basis
qc.measure(q[1], c[1])
# Measure qubit (1) in the |+> and |-> basis
qc.h(q[0])
qc.measure(q[0], c[0])
# If needed Perform a phase correction to qubit (3)
if c[0] == 1:
    qc.z(q[2])
# If needed Perform a bit flip correction to qubit (3)
if c[1] == 1:
    qc.x(q[2])
qc.measure(q[2], c[2])
# Compile and run the Quantum circuit on a simulator backend
job_sim = execute(qc, Q_simulator)
sim_result = job_sim.result()
M = sim_result.get_counts(qc)
print(M)
qc.draw()

{'111': 280, '001': 253, '110': 226, '000': 265}


░      ┌───┐┌─┐
q13_0: ───────────░───■──┤ H ├┤M├
       ┌───┐      ░ ┌─┴─┐└┬─┬┘└╥┘
q13_1: ┤ H ├──■───░─┤ X ├─┤M├──╫─
       └───┘┌─┴─┐ ░ └┬─┬┘ └╥┘  ║ 
q13_2: ─────┤ X ├─░──┤M├───╫───╫─
            └───┘ ░  └╥┘   ║   ║ 
 c2: 3/═══════════════╩════╩═══╩═
                      2    1   0

EXERCISE: Edit this cell to describe the _meaning_ of the qubits measured above.  Can you identify the input qubit, the Bell pair used for the teleportation, and the output qubit (which is hopefully the same as the input qubit)? Hint: you may want to edit the code above and add some additional print or measurement operations, or use `print(oq.Wavefunction(qc))` to learn more about the wavefunction at different points in the execution.

## References:
---
1. Qiskit. “Qiskit/Qiskit-Tutorials.” GitHub, 20 July 2019, github.com/Qiskit/qiskit-tutorials.
2. Koch, Daniel, Laura Wessing, and Paul M. Alsing. "Introduction to Coding Quantum Algorithms: A Tutorial Series Using Pyquil." arXiv preprint arXiv:1903.05195 (2019).
3. stevenatkin. “Untangling Quantum Teleportation.” Medium, Qiskit, 24 July 2018, medium.com/qiskit/untangling-quantum-teleportation-919cbd673074.
4. All pictures used are from Google image search
